In [1]:
%pip install scikit-learn pandas numpy matplotlib tabulate


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python3.8 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np
import re
pd.set_option("display.max_columns", None)
tf.get_logger().setLevel('INFO')
%cd /mnt

2023-08-04 19:13:22.088373: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-04 19:13:22.653356: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


/mnt


In [2]:
def unzip(path_to_zip_file, directory_to_extract_to):
    """
    Args:
        path_to_zip_file (str): 압축파일 경로
        directory_to_extract_to (str): 압축 풀고 저장할 경로
    """
    import zipfile
    with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
        zip_ref.extractall(directory_to_extract_to)


In [3]:
data = pd.read_csv("Full-Dataset/processed/url_encoded_dataset.csv").sample(frac=1)
data = data.drop(["Unnamed: 0", "http.request.uri.query", "http.file_data", "http.request.full_uri", 
                  "tcp.options", "http.request.uri.path", "http.request.uri.query.dict", "http.file_data.dict", "http.request.uri.scheme"], axis="columns")
data["http.data"] = data["http.data"].fillna("")
original_data = data.copy()

# RNN학습시, URL정보를 바탕으로 분류를 진행할 수 있으므로 URL 정보를 삭제해야함.
data["tcp.payload"] = original_data["tcp.payload"].astype("str").apply(lambda x: " ".join([_ for _ in x.split(" ") if not (_.startswith("/DVWA") and _.endswith(".php"))]))

/tmp/ipykernel_352/524550114.py:1: DtypeWarning: Columns (10,14,19,27) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("Full-Dataset/processed/url_encoded_dataset.csv").sample(frac=1)


## RNN을 위한 전처리

In [4]:
# Tokenize values
tokenizer = Tokenizer(oov_token="<NUMBER>", filters='!"#$%&()*+,-./:?@[\\]^_`{|}~\t\n=',)
tokenizer.fit_on_texts(original_data["http.data"])
padded = pad_sequences(tokenizer.texts_to_sequences(original_data["http.data"]), padding="post", value=0)
data["http.data"] = padded.tolist()

In [5]:
word_index_dict = {val: key for key, val in tokenizer.word_index.items()}
pd.Series(tokenizer.texts_to_sequences(original_data["http.data"])).apply(lambda x: ", ".join([(word_index_dict)[_] for _ in x])).unique()

array(['username, admin, password, 00, login, login', '',
       '0, 0, id, 8, 20and, 208183, 3dcast, 28, 28chr, 28113, 29, 7c, 7cchr, 28120, 29, 7c, 7cchr, 28122, 29, 7c, 7cchr, 28113, 29, 7c, 7cchr, 28113, 29, 29, 7c, 7c, 28select, 20, 28case, 20when, 20, 288183, 3d8183, 29, 20then, 201, 20else, 200, 20end, 29, 29, 3a, 3atext, 7c, 7c, 28chr, 28113, 29, 7c, 7cchr, 2898, 29, 7c, 7cchr, 28107, 29, 7c, 7cchr, 28118, 29, 7c, 7cchr, 28113, 29, 29, 20as, 20numeric, 29, 20kakw, submit, submitid, 58',
       ..., '0, 0, id, 9, submit, submitid, 3d59, 20and, 209385, 3d8004',
       '0, 0, id, 8, 20order, 20by, 201, 20fmlu, submit, submitid, 58',
       '0, 0, id, 2, submit, submitid, 3d53, 27, 29, 20waitfor, 20delay, 20, 270, 3a0, 3a5, 27, 20and, 20, 28, 27seva, 27, 3d, 27seva'],
      dtype=object)

In [6]:
# Tokenize tcp.payload
tokenizer_payload = Tokenizer(oov_token="<PAD>")
tokenizer_payload.fit_on_texts(original_data["tcp.payload"].astype("str").apply(lambda x: " ".join([x.split(" ")[0],*x.split(" ")[2:]]))) # url 정보 제거
padded = pad_sequences(tokenizer_payload.texts_to_sequences(original_data["tcp.payload"].astype("str")), padding="post", value=0)
data["tcp.payload"] = padded.tolist()

In [7]:
data.head()

,tcp.len,tcp.checksum,tcp.ack,http.response,tcp.connection.fin,ip.dst_host,http.content_length,Attack_type,arp.opcode,tcp.srcport,tcp.payload,arp.dst.proto_ipv4,tcp.connection.synack,tcp.seq,http.referer,frame.time,tcp.dstport,tcp.connection.syn,tcp.connection.rst,ip.src_host,tcp.ack_raw,tcp.flags,arp.src.proto_ipv4,tcp.flags.ack,arp.hw.size,0,GET,POST,TRACE,HTTP/1.0,HTTP/1.1,http.request.uri.netloc,http.data
63052,261.0,0x0000d93c,1.0,0.0,0.0,192.168.0.128,38.0,Password,0.0,34990.0,"[28, 1, 27, 1, 5, 4, 2, 9, 6, 7, 2, 8, 10, 11,...",0,0.0,1.0,0.0,2021 20:20:24.965807000,80.0,0.0,0.0,192.168.0.170,3.814847e+09,0x00000018,0,1.0,0.0,False,False,True,False,True,False,192.168.0.128,"[5, 6, 7, 8, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
23526,86.0,0x0000ac99,1.0,0.0,0.0,192.168.0.128,0.0,Password,0.0,35828.0,"[15, 1, 27, 1, 5, 4, 2, 9, 6, 7, 2, 8, 10, 11,...",0,0.0,1.0,0.0,2021 18:01:53.845631000,80.0,0.0,0.0,192.168.0.170,2.396583e+09,0x00000018,0,1.0,0.0,False,True,False,False,True,False,192.168.0.128,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
75140,616.0,0x0000313e,1.0,0.0,0.0,192.168.0.128,0.0,SQL_injection,0.0,45264.0,"[15, 1, 1, 1, 1, 1, 45, 1, 1, 1, 120, 1, 1, 69...",0,0.0,1.0,0.0,2021 21:43:15.733881000,80.0,0.0,0.0,192.168.0.170,3.373795e+09,0x00000018,0,1.0,0.0,False,True,False,False,False,True,192.168.0.128,"[3, 3, 13, 81, 16, 476, 121, 60, 29, 12, 4, 9,..."
23634,86.0,0x000030a8,1.0,0.0,0.0,192.168.0.128,0.0,Password,0.0,36136.0,"[15, 1, 27, 1, 5, 4, 2, 9, 6, 7, 2, 8, 10, 11,...",0,0.0,1.0,0.0,2021 18:02:22.632152000,80.0,0.0,0.0,192.168.0.170,4.176040e+08,0x00000018,0,1.0,0.0,False,True,False,False,True,False,192.168.0.128,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
69731,86.0,0x000061ab,1.0,0.0,0.0,192.168.0.128,0.0,Password,0.0,53956.0,"[15, 1, 27, 1, 5, 4, 2, 9, 6, 7, 2, 8, 10, 11,...",0,0.0,1.0,0.0,2021 20:42:40.117183000,80.0,0.0,0.0,192.168.0.170,3.952369e+09,0x00000018,0,1.0,0.0,False,True,False,False,True,False,192.168.0.128,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [8]:
original_data.head()

,tcp.len,tcp.checksum,tcp.ack,http.response,tcp.connection.fin,ip.dst_host,http.content_length,Attack_type,arp.opcode,tcp.srcport,tcp.payload,arp.dst.proto_ipv4,tcp.connection.synack,tcp.seq,http.referer,frame.time,tcp.dstport,tcp.connection.syn,tcp.connection.rst,ip.src_host,tcp.ack_raw,tcp.flags,arp.src.proto_ipv4,tcp.flags.ack,arp.hw.size,0,GET,POST,TRACE,HTTP/1.0,HTTP/1.1,http.request.uri.netloc,http.data
63052,261.0,0x0000d93c,1.0,0.0,0.0,192.168.0.128,38.0,Password,0.0,34990.0,POST /DVWA/login.php HTTP/1.0\nHost: 192.168.0...,0,0.0,1.0,0.0,2021 20:20:24.965807000,80.0,0.0,0.0,192.168.0.170,3.814847e+09,0x00000018,0,1.0,0.0,False,False,True,False,True,False,192.168.0.128,username=admin&password=00&Login=Login
23526,86.0,0x0000ac99,1.0,0.0,0.0,192.168.0.128,0.0,Password,0.0,35828.0,GET /DVWA/login.php HTTP/1.0\nHost: 192.168.0....,0,0.0,1.0,0.0,2021 18:01:53.845631000,80.0,0.0,0.0,192.168.0.170,2.396583e+09,0x00000018,0,1.0,0.0,False,True,False,False,True,False,192.168.0.128,
75140,616.0,0x0000313e,1.0,0.0,0.0,192.168.0.128,0.0,SQL_injection,0.0,45264.0,GET /DVWA/vulnerabilities/sqli_blind/?id=8%20A...,0,0.0,1.0,0.0,2021 21:43:15.733881000,80.0,0.0,0.0,192.168.0.170,3.373795e+09,0x00000018,0,1.0,0.0,False,True,False,False,False,True,192.168.0.128,0.0&id=8%20AND%208183%3DCAST%28%28CHR%28113%29...
23634,86.0,0x000030a8,1.0,0.0,0.0,192.168.0.128,0.0,Password,0.0,36136.0,GET /DVWA/login.php HTTP/1.0\nHost: 192.168.0....,0,0.0,1.0,0.0,2021 18:02:22.632152000,80.0,0.0,0.0,192.168.0.170,4.176040e+08,0x00000018,0,1.0,0.0,False,True,False,False,True,False,192.168.0.128,
69731,86.0,0x000061ab,1.0,0.0,0.0,192.168.0.128,0.0,Password,0.0,53956.0,GET /DVWA/login.php HTTP/1.0\nHost: 192.168.0....,0,0.0,1.0,0.0,2021 20:42:40.117183000,80.0,0.0,0.0,192.168.0.170,3.952369e+09,0x00000018,0,1.0,0.0,False,True,False,False,True,False,192.168.0.128,


In [9]:
def zipFile(fileName):
    import shutil
    shutil.make_archive(fileName,"zip", fileName)

## DNN을 위한 전처리

In [10]:
# process time data into hour
def time_to_sec(time):
    hour, minute, second = list(map(float, time[5:].strip().split(":")))
    sec = hour*3600 + minute*60 + second
    return sec / 3600
data["frame.time"] = original_data["frame.time"].apply(time_to_sec)

In [11]:
# tcp.checksum float로 변경
data["tcp.checksum"] = original_data["tcp.checksum"].astype("str").apply(lambda x: int(x[2:] if x[2:] else "0", 16))

In [12]:
# tcp.flag float로 변경
data["tcp.flags"] = original_data["tcp.flags"].astype("str").apply(lambda x: int(x[2:] if x[2:] else "0", 16))

In [13]:
# tcp.ack_raw 정규화
data["tcp.ack_raw"] = (original_data["tcp.ack_raw"] - original_data["tcp.ack_raw"].mean()) / original_data["tcp.ack_raw"].std()

In [14]:
# tcp.ack 정규화
data["tcp.ack"] = (original_data["tcp.ack"] - original_data["tcp.ack"].mean()) / original_data["tcp.ack"].std()

In [15]:
for feature in ["GET", "POST", "TRACE", "HTTP/1.0", "HTTP/1.1"]:
    data[feature] = original_data[feature].apply(int)

In [16]:
# http.referer는 [0.0, '0.0', '0', '127.0.0.1'] 의 값만을 갖고 있으므로, 의미가 없음.
data = data.drop(["http.referer"], axis="columns") 

In [17]:
data = data.drop(["0"], axis="columns")

In [18]:
# DNN을 할 columns
DNN_COLS = ['frame.time', 'tcp.checksum', 'http.response',
       'http.content_length',
       'tcp.connection.rst', 'arp.hw.size', 'tcp.len',
       'tcp.seq', 'tcp.connection.syn',
       'tcp.flags.ack', 'tcp.ack_raw', 'tcp.connection.fin', 'tcp.dstport',
       'tcp.flags', 'tcp.ack', 'tcp.connection.synack', 
       'tcp.srcport', 'arp.opcode', 'GET', 'POST', 'TRACE', 'HTTP/1.0',
       'HTTP/1.1']
data[DNN_COLS].head(10)

,frame.time,tcp.checksum,http.response,http.content_length,tcp.connection.rst,arp.hw.size,tcp.len,tcp.seq,tcp.connection.syn,tcp.flags.ack,tcp.ack_raw,tcp.connection.fin,tcp.dstport,tcp.flags,tcp.ack,tcp.connection.synack,tcp.srcport,arp.opcode,GET,POST,TRACE,HTTP/1.0,HTTP/1.1
63052,20.340268,55612,0.0,38.0,0.0,0.0,261.0,1.0,0.0,1.0,1.355915,0.0,80.0,24,-0.263640,0.0,34990.0,0.0,0,1,0,1,0
23526,18.031624,44185,0.0,0.0,0.0,0.0,86.0,1.0,0.0,1.0,0.241371,0.0,80.0,24,-0.263640,0.0,35828.0,0.0,1,0,0,1,0
75140,21.721037,12606,0.0,0.0,0.0,0.0,616.0,1.0,0.0,1.0,1.009313,0.0,80.0,24,-0.263640,0.0,45264.0,0.0,1,0,0,0,1
23634,18.039620,12456,0.0,0.0,0.0,0.0,86.0,1.0,0.0,1.0,-1.313809,0.0,80.0,24,-0.263640,0.0,36136.0,0.0,1,0,0,1,0
69731,20.711144,25003,0.0,0.0,0.0,0.0,86.0,1.0,0.0,1.0,1.463986,0.0,80.0,24,-0.263640,0.0,53956.0,0.0,1,0,0,1,0
81420,18.327945,40260,0.0,0.0,0.0,0.0,0.0,442.0,0.0,1.0,0.108078,0.0,59578.0,16,1.562255,0.0,80.0,0.0,0,0,0,0,0
16712,17.583878,38513,0.0,0.0,0.0,0.0,221.0,1.0,0.0,1.0,-1.625604,0.0,80.0,24,-0.263640,0.0,44722.0,0.0,1,0,0,1,0
17667,17.636561,1162,0.0,0.0,0.0,0.0,86.0,1.0,0.0,1.0,1.375958,0.0,80.0,24,-0.263640,0.0,47420.0,0.0,1,0,0,1,0
31076,18.554754,18732,0.0,0.0,0.0,0.0,221.0,1.0,0.0,1.0,0.263194,0.0,80.0,24,-0.263640,0.0,57258.0,0.0,1,0,0,1,0
9046,17.159658,2187,0.0,0.0,0.0,0.0,86.0,1.0,0.0,1.0,-1.052697,0.0,80.0,24,-0.263640,0.0,51186.0,0.0,1,0,0,1,0


## 데이터 분할

In [19]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.3)
datas = [train_test_split(data, test_size=0.3) for _ in range(10)]

In [ ]:
datas_ = []
def to_onehot_wrapper(labels):
    def to_onehot(x):
        l = [0 for _ in range(labels.__len__())]
        l[labels[x]] = 1
        return l
    return to_onehot
for idx, (train, test) in enumerate(datas):
    labels = {key: i for i, key in enumerate(sorted(test["Attack_type"].unique()))}
    test_X, test_Y = test.drop(["Attack_type"], axis="columns"), np.stack(test["Attack_type"].apply(to_onehot_wrapper(labels)))
    train_X, train_Y =  train.drop(["Attack_type"], axis="columns"), np.stack(train["Attack_type"].apply(to_onehot_wrapper(labels)))
    datas_.append([test_X, test_Y, train_X, train_Y])


In [21]:
labels = {key: i for i, key in enumerate(sorted(test["Attack_type"].unique()))}
def to_onehot(x):
    l = [0 for _ in range(labels.__len__())]
    l[labels[x]] = 1
    return l
test_X, test_Y = test.drop(["Attack_type"], axis="columns"), np.stack(test["Attack_type"].apply(to_onehot))
train_X, train_Y =  train.drop(["Attack_type"], axis="columns"), np.stack(train["Attack_type"].apply(to_onehot))

## RNN 실행

In [107]:
import keras.backend as K
def f1_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val
def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision
def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


In [ ]:
payload_test_X ,RNN_test_X =  np.stack(test["tcp.payload"]),np.stack(test["http.data"])
payload_train_X ,RNN_train_X =  np.stack(train["tcp.payload"]),np.stack(train["http.data"])

payload_test_Xs = []
RNN_test_Xs = []
payload_train_Xs = []
RNN_train_Xs = []
for idx, _dt in enumerate(datas):
    _train, _test = _dt
    print(f"{_train.shape=}, {_test.shape=}")
    payload_test_Xs.append(
        np.stack(_test["tcp.payload"])
    ) 
    RNN_test_Xs.append(
        np.stack(_test["http.data"])
    ) 
    
    payload_train_Xs.append(
        np.stack(_train["tcp.payload"])
    ) 
    RNN_train_Xs.append(
        np.stack(_train["http.data"])
    ) 
    

In [52]:
file_data_vocab_size = tokenizer.word_index.keys().__len__() + 1
file_data_input_dim = RNN_train_X.shape[1]

In [62]:
import os

layers = [
            keras.layers.Embedding(file_data_vocab_size, 128, input_length=file_data_input_dim),
            keras.layers.LSTM(128, dropout=0.7, activation="tanh"),
            keras.layers.Dense(100, activation=tf.nn.relu),
            keras.layers.Dense(4, activation='softmax'),
        ]

compileArgs = {
    "loss": tf.keras.losses.CategoricalCrossentropy(), 
    "optimizer": tf.keras.optimizers.Adam(learning_rate=.005), 
    "metrics": ["accuracy", f1_score, precision, recall],
}
RNN_models = []
for idx, (RNN_train_X, (_, _, _, train_Y)) in enumerate(zip(RNN_train_Xs, datas_)):
    RNN_model = keras.Sequential(
        layers = [
            keras.layers.Embedding(file_data_vocab_size, 128, input_length=file_data_input_dim),
            keras.layers.LSTM(128, dropout=0.7, activation="tanh"),
            keras.layers.Dense(100, activation=tf.nn.relu),
            keras.layers.Dense(4, activation='softmax'),
        ]
    )
    RNN_model.compile(
        **compileArgs
    )
    print(f"repeat no.{idx+1}")
    RNN_model.fit(RNN_train_X, train_Y, epochs=15, batch_size=128)
    RNN_models.append(RNN_model)
    # RNN_model.save("http_data_RNN.model")
    # zipFile("http_data_RNN.model")

Epoch 1/15
510/510 [==============================] - 17s 32ms/step - loss: 0.5681 - accuracy: 0.8054 - f1_score: 0.8037 - precision: 0.8037 - recall: 0.8037
Epoch 2/15
510/510 [==============================] - 7s 13ms/step - loss: 0.5268 - accuracy: 0.8131 - f1_score: 0.8131 - precision: 0.8132 - recall: 0.8131
Epoch 3/15
510/510 [==============================] - 6s 12ms/step - loss: 0.5089 - accuracy: 0.8202 - f1_score: 0.8201 - precision: 0.8203 - recall: 0.8199
Epoch 4/15
510/510 [==============================] - 6s 12ms/step - loss: 0.3707 - accuracy: 0.8405 - f1_score: 0.8405 - precision: 0.8405 - recall: 0.8405
Epoch 5/15
510/510 [==============================] - 6s 11ms/step - loss: 0.3673 - accuracy: 0.8414 - f1_score: 0.8413 - precision: 0.8413 - recall: 0.8413
Epoch 6/15
510/510 [==============================] - 6s 11ms/step - loss: 0.3670 - accuracy: 0.8413 - f1_score: 0.8414 - precision: 0.8414 - recall: 0.8414
Epoch 7/15
510/510 [==============================] - 6s 

In [108]:
payload_vocab_size = tokenizer_payload.word_index.keys().__len__() + 1
payload_input_dim = payload_train_X.shape[1]

In [109]:
layers = [
            keras.layers.Embedding(payload_vocab_size, 128, input_length=payload_input_dim),
            keras.layers.LSTM(128, dropout=0.7),
            keras.layers.Dense(100, activation="relu"),
            keras.layers.Dense(4, activation='softmax'),
        ]
payload_RNN_model_args = {
    "loss": tf.keras.losses.CategoricalCrossentropy(), 
    "optimizer": "adam", 
    "metrics": ["accuracy", f1_score, precision, recall],
}
payload_models = []
for idx, (payload_train_X, (_, _, _, train_Y)) in enumerate(zip(payload_train_Xs, datas_)):
    payload_RNN_model = keras.Sequential(layers = [
            keras.layers.Embedding(payload_vocab_size, 128, input_length=payload_input_dim),
            keras.layers.LSTM(128, dropout=0.7),
            keras.layers.Dense(100, activation="relu"),
            keras.layers.Dense(4, activation='softmax'),
        ])
    payload_RNN_model.compile(
        **payload_RNN_model_args
    )
    print(f"repeat no.{idx+1}")
    payload_RNN_model.fit(payload_train_X, train_Y, epochs=15, batch_size=128)
    payload_models.append(payload_RNN_model)


repeat no.1
Epoch 1/15
510/510 [==============================] - 13s 23ms/step - loss: 0.6160 - accuracy: 0.7899 - f1_score: 0.7810 - precision: 0.7819 - recall: 0.7802
Epoch 2/15
510/510 [==============================] - 7s 14ms/step - loss: 0.4683 - accuracy: 0.8246 - f1_score: 0.8245 - precision: 0.8246 - recall: 0.8245
Epoch 3/15
510/510 [==============================] - 7s 14ms/step - loss: 0.5133 - accuracy: 0.8166 - f1_score: 0.8165 - precision: 0.8167 - recall: 0.8164
Epoch 4/15
510/510 [==============================] - 7s 14ms/step - loss: 0.5306 - accuracy: 0.8426 - f1_score: 0.8422 - precision: 0.8427 - recall: 0.8417
Epoch 5/15
510/510 [==============================] - 6s 13ms/step - loss: 0.6050 - accuracy: 0.7960 - f1_score: 0.7964 - precision: 0.7976 - recall: 0.7953
Epoch 6/15
510/510 [==============================] - 6s 12ms/step - loss: 0.5338 - accuracy: 0.8197 - f1_score: 0.8198 - precision: 0.8202 - recall: 0.8196
Epoch 7/15
510/510 [=========================

In [80]:
file_data_predictions = []
for RNN_model, RNN_test_X in zip(RNN_models, RNN_test_Xs):
    file_data_predictions.append(RNN_model.predict(RNN_test_X))

  1/875 [..............................] - ETA: 16s

875/875 [==============================] - 2s 3ms/step


In [110]:
payload_predictions = []
for payload_RNN_model, payload_test_X in zip(payload_models, payload_test_Xs):
    payload_predictions.append(RNN_model.predict(payload_test_X))

  1/875 [..............................] - ETA: 16s

875/875 [==============================] - 3s 3ms/step


In [111]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix, f1_score

In [83]:
file_data_confusion_matrixs = []
for file_data_prediction, (_, test_Y, _, _) in zip(file_data_predictions, datas_):
    p = pd.DataFrame(confusion_matrix(np.argmax(test_Y, axis=1), np.argmax(file_data_prediction, axis=1)),index="TRUE_"+pd.Series(labels.keys()), columns="PRED_"+pd.Series(labels.keys()))
    file_data_confusion_matrixs.append(p)

In [112]:
payload_confusion_matrixs = []
for payload_prediction, (_, test_Y, _, _) in zip(payload_predictions, datas_):
    p = \
    pd.DataFrame(confusion_matrix(np.argmax(test_Y, axis=1), np.argmax(payload_prediction, axis=1)),index="TRUE_"+pd.Series(labels.keys()), columns="PRED_"+pd.Series(labels.keys()))
    payload_confusion_matrixs.append(p)

## DNN 실행

In [99]:
import keras.backend as K
def f1_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val
def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision
def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


In [100]:
DNN_layers = [
        keras.layers.Dense(4196, activation="relu"),
        keras.layers.Dense(512, activation="relu"),
        keras.layers.Dense(512, activation="relu"),
        keras.layers.Dense(512, activation="relu"),
        keras.layers.Dense(512, activation="relu"),
        keras.layers.Dense(512, activation="relu"),
        keras.layers.Dense(512, activation="relu"),
        keras.layers.Dense(len(labels.keys()), activation="softmax"),
    ]
DNN_compile = {
    "loss": tf.keras.losses.CategoricalCrossentropy(), 
    "optimizer": "adam", 
    "metrics": ["accuracy", f1_score, precision, recall],
}
# predictions = []
DNN_models = []
DNN_predictions = []
for idx, (test_X, test_Y, train_X, train_Y) in enumerate(datas_):
    DNN_model = keras.Sequential(layers=DNN_layers.copy())
    DNN_model.compile(**DNN_compile)
    print(f"repeat no.{idx+1}")
    DNN_model.fit(train_X[DNN_COLS], train_Y, epochs=15, batch_size=64)
    DNN_prediction = DNN_model.predict(test_X[DNN_COLS])
    DNN_predictions.append(DNN_prediction)

repeat no.1
Epoch 1/15
1020/1020 [==============================] - 6s 5ms/step - loss: 14.2825 - accuracy: 0.8270 - f1_score: 0.8248 - precision: 0.8297 - recall: 0.8206
Epoch 2/15
1020/1020 [==============================] - 5s 5ms/step - loss: 0.3220 - accuracy: 0.9130 - f1_score: 0.9129 - precision: 0.9147 - recall: 0.9113
Epoch 3/15
1020/1020 [==============================] - 5s 5ms/step - loss: 0.2471 - accuracy: 0.9337 - f1_score: 0.9336 - precision: 0.9346 - recall: 0.9326
Epoch 4/15
1020/1020 [==============================] - 5s 5ms/step - loss: 0.2567 - accuracy: 0.9357 - f1_score: 0.9357 - precision: 0.9366 - recall: 0.9348
Epoch 5/15
1020/1020 [==============================] - 5s 5ms/step - loss: 0.2128 - accuracy: 0.9448 - f1_score: 0.9447 - precision: 0.9454 - recall: 0.9439
Epoch 6/15
1020/1020 [==============================] - 5s 5ms/step - loss: 0.1867 - accuracy: 0.9508 - f1_score: 0.9507 - precision: 0.9513 - recall: 0.9502
Epoch 7/15
1020/1020 [=================

In [103]:
DNN_confusion_matrix = []

for prediction, (test_X, test_Y, train_X, train_Y) in zip(DNN_predictions, datas_):
    DNN_confusion_matrix.append(pd.DataFrame(confusion_matrix(np.argmax(test_Y, axis=1), np.argmax(prediction, axis=1)), index="TRUE_"+pd.Series(labels.keys()), columns="PRED_"+pd.Series(labels.keys())))
    # .to_markdown(tablefmt="grid")

In [104]:
# ensemble model
result_list = []
result_weight_list = []
for DNN_prediction, file_data_prediction in zip(DNN_predictions, file_data_predictions):
    result = np.array([([d1,d2,d3,d4] if max([d1,d2,d3,d4])==d4 else [p1, p2 ,p3, p4] if max([p1, p2, p3, p4]) in [p2, p3, p4] else [d1, 0, 0, d4]) for (d1,d2,d3,d4),(p1,p2,p3,p4) in zip(DNN_prediction, file_data_prediction)])
    result_weight = np.array([([d1,d2,d3,d4] if max([d1,d2,d3,d4])==d4 else [p1, p2 ,p3, p4] if max([p1, p2, p3, p4]) in [p2, p3, p4] else [d1, 0, 0, d4*2]) for (d1,d2,d3,d4),(p1,p2,p3,p4) in zip(DNN_prediction, file_data_prediction)])
    result_list.append(result)
    result_weight_list.append(result_weight)

In [113]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix, f1_score
result_confusion_matrix_list = []
result_weight_confusion_matrix_list = []
for (test_X, test_Y, train_X, train_Y), (result, result_weight) in zip(datas_,zip(result_list, result_weight_list)):
    print("result - default")
    precision_Score, recall_Score, f1_Score = precision_score(np.argmax(test_Y, axis=1), np.argmax(result, axis=1), average="macro"),\
        recall_score(np.argmax(test_Y, axis=1), np.argmax(result, axis=1), average="macro"), \
            f1_score(np.argmax(test_Y, axis=1), np.argmax(result, axis=1), average="macro")
            
    print(f"""{precision_Score=}\t{recall_Score=}\t{f1_Score=}""")
    result_confusion_matrix = pd.DataFrame(confusion_matrix(np.argmax(test_Y, axis=1), np.argmax(result, axis=1)), index="TRUE_"+pd.Series(labels.keys()), columns="PRED_"+pd.Series(labels.keys()))
    print(
    result_confusion_matrix.to_markdown(tablefmt="grid")
    )
    print()
    print()
    print("result-weighted")
    precision_Score, recall_Score, f1_Score = precision_score(np.argmax(test_Y, axis=1), np.argmax(result_weight, axis=1), average="macro"),\
        recall_score(np.argmax(test_Y, axis=1), np.argmax(result_weight, axis=1), average="macro"), \
            f1_score(np.argmax(test_Y, axis=1), np.argmax(result_weight, axis=1), average="macro")
            
    print(f"""{precision_Score=}\t{recall_Score=}\t{f1_Score=}""")
    result_weight_confusion_matrix = pd.DataFrame(confusion_matrix(np.argmax(test_Y, axis=1), np.argmax(result_weight, axis=1)), index="TRUE_"+pd.Series(labels.keys()), columns="PRED_"+pd.Series(labels.keys()))
    print(
    result_weight_confusion_matrix.to_markdown(tablefmt="grid")
    )
    result_confusion_matrix_list.append(result_confusion_matrix)
    result_weight_confusion_matrix_list.append(result_weight_confusion_matrix)

result - default
precision_Score=0.9959857887694367	recall_Score=0.9817456986991188	f1_Score=0.9885036535610683
+--------------------+-----------------+----------------------+------------------+------------+
|                    |   PRED_Password |   PRED_SQL_injection |   PRED_Uploading |   PRED_XSS |
+====================+=================+======================+==================+============+
| TRUE_Password      |           21325 |                    0 |                0 |          0 |
+--------------------+-----------------+----------------------+------------------+------------+
| TRUE_SQL_injection |               0 |                 1322 |                0 |          0 |
+--------------------+-----------------+----------------------+------------------+------------+
| TRUE_Uploading     |               0 |                    0 |              558 |          0 |
+--------------------+-----------------+----------------------+------------------+------------+
| TRUE_XSS           |  

precision_Score=0.9959857887694367	recall_Score=0.9817456986991188	f1_Score=0.9885036535610683
+--------------------+-----------------+----------------------+------------------+------------+
|                    |   PRED_Password |   PRED_SQL_injection |   PRED_Uploading |   PRED_XSS |
+====================+=================+======================+==================+============+
| TRUE_Password      |           21325 |                    0 |                0 |          0 |
+--------------------+-----------------+----------------------+------------------+------------+
| TRUE_SQL_injection |               0 |                 1322 |                0 |          0 |
+--------------------+-----------------+----------------------+------------------+------------+
| TRUE_Uploading     |               0 |                    0 |              558 |          0 |
+--------------------+-----------------+----------------------+------------------+------------+
| TRUE_XSS           |             348 | 

In [114]:
for name, _matrix in \
    [
        ["DNN", DNN_confusion_matrix], # DNN 
        ["result_confusion_matrix", result_confusion_matrix_list], # DNN + file_data(user form)
        ["result_weight_confusion_matrix", result_weight_confusion_matrix_list], # DNN + file_data(user form) modified
        ["file_data", file_data_confusion_matrixs], # file_data(user_form) LSTM
        ["payload", payload_confusion_matrixs], # payload_data LSTM
    ]:
    print("-"*60)
    print(name)
    print("-"*60)
    for _ in _matrix:
        print(_.to_markdown(tablefmt="grid"))
        print()
    print()

------------------------------------------------------------
DNN
------------------------------------------------------------
+--------------------+-----------------+----------------------+------------------+------------+
|                    |   PRED_Password |   PRED_SQL_injection |   PRED_Uploading |   PRED_XSS |
+====================+=================+======================+==================+============+
| TRUE_Password      |           21325 |                    0 |                0 |          0 |
+--------------------+-----------------+----------------------+------------------+------------+
| TRUE_SQL_injection |               7 |                 1315 |                0 |          0 |
+--------------------+-----------------+----------------------+------------------+------------+
| TRUE_Uploading     |             558 |                    0 |                0 |          0 |
+--------------------+-----------------+----------------------+------------------+------------+
| TRUE_XSS

In [176]:
i = 0
confusion_matrix = DNN_confusion_matrix
# _ 는 TRUE_ 이다.
# _.replace("TRUE_", "PRED_") 는 PRED 이다.
# TP: confusion_matrix[i].loc[_][_.replace("TRUE_", "PRED_")]
# TN: confusion_matrix[i][_.replace("TRUE_", "PRED_")].loc[_]
# FP: confusion_matrix[i].loc[_][_.replace("TRUE_", "PRED_")]
# FN: confusion_matrix[i].loc[_][_.replace("TRUE_", "PRED_")]
def index_not(df, index_name):
    return df.loc[df.index.str[-5:]!=index_name]
def column_not(df, column_name):
    return df.loc[:,[_ for _ in df.columns if _!=column_name]]

def TP(df, true_col_name):
    index_name = true_col_name.replace("TRUE_", "PRED_")
    return df.loc[true_col_name][index_name]

def TN(df, true_col_name):
    index_name = true_col_name.replace("TRUE_", "PRED_")
    return column_not(index_not(df, index_name), true_col_name).sum()

def FP(df, true_col_name):
    index_name = true_col_name.replace("TRUE_", "PRED_")
    return index_not(df.loc[true_col_name], index_name)

def FN(df, true_col_name):
    index_name = true_col_name.replace("TRUE_", "PRED_")
    return column_not(df, true_col_name)[index_name]
def accuracy(df, _):
    divisor = (TP(df, _).sum() + TN(df, _).sum()) 
    dividor = (TP(df, _).sum() + TN(df, _).sum() + FN(df, _).sum() + FP(df, _).sum())
    return divisor / (dividor)
precision = lambda df, _: TP(df, _).sum() / ((TP(df, _).sum() + FP(df, _).sum()) or 1)
recall = lambda df, _: TP(confusion_matrix[i], _).sum() / ((TP(confusion_matrix[i], _).sum() + FN(confusion_matrix[i], _).sum()) or 1)
def f1score(df, _):
    _precision = precision(df, _)
    _recall = recall(df, _)
    return 2 * (_precision * _recall) / ((_precision + _recall) or 1) 


# confusion_matrix_processData = {f"{name}": {f"{i}": {
#             "accuracy":{_.replace("TRUE_",""): accuracy(confusion_matrix[i], _) for _ in confusion_matrix[i].index},
#             "precision":{_.replace("TRUE_",""): precision(confusion_matrix[i], _) for _ in confusion_matrix[i].index},
#             "recall":{_.replace("TRUE_",""): recall(confusion_matrix[i], _) for _ in confusion_matrix[i].index},
#             "f1score":{_.replace("TRUE_",""): f1score(confusion_matrix[i], _) for _ in confusion_matrix[i].index},
#         }} for i in range(len(confusion_matrix)) for name, confusion_matrix in [
#             ["file_data_LSTM", file_data_confusion_matrixs], # file_data(user_form) LSTM
#             ["payload_LSTM", payload_confusion_matrixs], # payload_data LSTM
#             ["DNN", DNN_confusion_matrix], # DNN 
#             ["non_weighted_Ensemble", result_confusion_matrix_list], # DNN + file_data(user form)
#             ["weighted_Ensemble", result_weight_confusion_matrix_list], # DNN + file_data(user form) modified
#         ]
# }

confusion_matrix_processData = dict(
    [
        (f"{name}", {
            f"{i}": {
                "accuracy":{_.replace("TRUE_",""): accuracy(confusion_matrix[i], _) for _ in confusion_matrix[i].index},
                "precision":{_.replace("TRUE_",""): precision(confusion_matrix[i], _) for _ in confusion_matrix[i].index},
                "recall":{_.replace("TRUE_",""): recall(confusion_matrix[i], _) for _ in confusion_matrix[i].index},
                "f1score":{_.replace("TRUE_",""): f1score(confusion_matrix[i], _) for _ in confusion_matrix[i].index},
            } for i in range(len(confusion_matrix))
        })  for name, confusion_matrix in [
            ["file_data_LSTM", file_data_confusion_matrixs], # file_data(user_form) LSTM
            ["payload_LSTM", payload_confusion_matrixs], # payload_data LSTM
            ["DNN", DNN_confusion_matrix], # DNN 
            ["non_weighted_Ensemble", result_confusion_matrix_list], # DNN + file_data(user form)
            ["weighted_Ensemble", result_weight_confusion_matrix_list], # DNN + file_data(user form) modified
        ]
    ]
)
import json
print(
    json.dumps(confusion_matrix_processData,indent=4)
)
with open("performance_data.json", "w") as f:
    f.write(json.dumps(confusion_matrix_processData,indent=4))


{
    "file_data_LSTM": {
        "0": {
            "accuracy": {
                "Password": 0.5115815691158156,
                "SQL_injection": 0.9172120111469455,
                "Uploading": 0.9623545285882948,
                "XSS": 0.8469543374899076
            },
            "precision": {
                "Password": 0.5,
                "SQL_injection": 0.5,
                "Uploading": 0.5,
                "XSS": 0.0687768659632669
            },
            "recall": {
                "Password": 0.4895321610578027,
                "SQL_injection": 0.499809958190802,
                "Uploading": 0.0,
                "XSS": 0.5
            },
            "f1score": {
                "Password": 0.4947107131257829,
                "SQL_injection": 0.49990496103402393,
                "Uploading": 0.0,
                "XSS": 0.12092064582617656
            }
        },
        "1": {
            "accuracy": {
                "Password": 0.5118960887575196,
                "SQ

In [180]:
import csv
with open("performance_data_type01.csv", "w") as f:
    csvwriter = csv.writer(f)
    csvwriter.writerow(["model", "attempt", "score_type", "PW_Injection", "SQLi", "File_Inclusion", "XSS"])
    for model_name, data1 in confusion_matrix_processData.items():
        for attempt, data2 in data1.items():
            attempt = int(attempt) + 1
            for score_type, data3 in data2.items():
                csvwriter.writerow([model_name, attempt, score_type, data3["Password"], data3["SQL_injection"], data3["Uploading"], data3["XSS"]]) 
with open("performance_data_type02.csv", "w") as f:
    csvwriter = csv.writer(f)
    csvwriter.writerow(["model", "attempt", "score_type", "attack_name"])
    attack_names = {"Password":"PW_Injection", "SQL_injection":"SQLi", "Uploading":"File_Inclusion", "XSS":"XSS"}
    for model_name, data1 in confusion_matrix_processData.items():
        for attempt, data2 in data1.items():
            attempt = int(attempt) + 1
            for score_type, data3 in data2.items():
                for attack_name, score in data3.items():
                    csvwriter.writerow([model_name, attempt, score_type, attack_names[attack_name],score]) 

In [ ]:
PassWord_XSS_test_Y = []
PassWord_XSS_fit_Y = []
for pred, test in zip(result, test_Y):
    if not (test[1] + test[2]):
        PassWord_XSS_test_Y.append(test)
        PassWord_XSS_fit_Y.append(pred)


In [ ]:
precision_score(np.argmax(PassWord_XSS_test_Y, axis=1), np.argmax(PassWord_XSS_fit_Y, axis=1), average="macro"),\
    recall_score(np.argmax(PassWord_XSS_test_Y, axis=1), np.argmax(PassWord_XSS_fit_Y, axis=1), average="macro"), \
        f1_score(np.argmax(PassWord_XSS_test_Y, axis=1), np.argmax(PassWord_XSS_fit_Y, axis=1), average="macro")

(0.990989587375283, 0.9592054226068734, 0.9741960846502298)